In [1]:
import pathlib
import random

import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
img_root = pathlib.Path("flower_photos")
all_img_path = list(img_root.glob('*/*'))
# cast to str
all_img_path = [str(onePath) for onePath in all_img_path]
random.shuffle(all_img_path)

len(all_img_path)

3670

In [3]:
labels = sorted(root_item.name for root_item in img_root.glob('*/') if root_item.is_dir())
labels

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [4]:
label_to_index = dict((name, index) for index, name in enumerate(labels))
label_to_index

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [5]:
all_image_index = [label_to_index[pathlib.Path(onepath).parent.name] for onepath in all_img_path]
all_image_index[:10]
# label_list = list(labels)
# for oneindex in all_image_index[:10]:
#     print(label_list[oneindex])

[4, 0, 2, 4, 4, 0, 3, 2, 3, 2]

In [6]:
def load_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0
    return image

In [7]:
path_ds = tf.data.Dataset.from_tensor_slices(all_img_path)
path_ds

<TensorSliceDataset shapes: (), types: tf.string>

In [8]:
# image_ds = tf.data.Dataset.from_tensor_slices([load_image(onepath) for onepath in all_img_path])

In [9]:
# index_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_index, tf.int64))

In [10]:
# image_index_ds = tf.data.Dataset.zip((image_ds, index_ds))

In [11]:
path_index_ds = tf.data.Dataset.from_tensor_slices((all_img_path, all_image_index))

def pathIndex_to_imageIndex(path, index):
    return load_image(path), index

image_index_ds = path_index_ds.map(pathIndex_to_imageIndex)

In [12]:
image_index_ds

<MapDataset shapes: ((192, 192, None), ()), types: (tf.float32, tf.int32)>

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds = image_index_ds.shuffle(buffer_size=len(all_img_path))
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# 当模型在训练的时候，`prefetch` 使数据集在后台取得 batch。
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

<PrefetchDataset shapes: ((None, 192, 192, None), (None,)), types: (tf.float32, tf.int32)>

In [14]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
mobile_net.trainable=False

In [15]:
def change_range(image, index):
  return 2*image-1, index

ds = ds.map(change_range)

In [16]:
image_batch, label_batch = next(iter(ds))
feature_map_batch = mobile_net(image_batch)
feature_map_batch.shape

TensorShape([32, 6, 6, 1280])

In [17]:
model = tf.keras.Sequential([
  mobile_net,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(labels), activation = 'softmax')])

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_192 (Model) (None, 6, 6, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [20]:
model_log = model.fit(ds, epochs=10, steps_per_epoch=3)

Train for 3 steps
Epoch 1/10
3/3 [==============================] - 13s 4s/step - loss: 1.5462 - accuracy: 0.3542
Epoch 2/10
3/3 [==============================] - 2s 814ms/step - loss: 1.4848 - accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 2s 787ms/step - loss: 1.3745 - accuracy: 0.3958
Epoch 4/10
3/3 [==============================] - 2s 778ms/step - loss: 1.2015 - accuracy: 0.4896
Epoch 5/10
3/3 [==============================] - 2s 787ms/step - loss: 1.1197 - accuracy: 0.5417
Epoch 6/10
3/3 [==============================] - 2s 785ms/step - loss: 1.0425 - accuracy: 0.6250
Epoch 7/10
3/3 [==============================] - 2s 827ms/step - loss: 0.9446 - accuracy: 0.6354
Epoch 8/10
3/3 [==============================] - 2s 798ms/step - loss: 1.0193 - accuracy: 0.6771
Epoch 9/10
3/3 [==============================] - 2s 780ms/step - loss: 0.9119 - accuracy: 0.6562
Epoch 10/10
3/3 [==============================] - 2s 774ms/step - loss: 0.8851 - accuracy: 0.7292


In [ ]:
import IPython.display as display

In [ ]:
def showImage(path):
    display.display(display.Image(path))

In [ ]:
for i in range(3):
    imgpath = random.choice(all_img_path)
    showImage(imgpath)